<a href="https://colab.research.google.com/github/Santosh3010/Advance-AI-Project/blob/main/AAI_Event_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.utils import resample
import string
#sklearn package 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn. preprocessing import LabelEncoder,StandardScaler
import sklearn.feature_selection

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import ComplementNB,MultinomialNB,GaussianNB 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import GradientBoostingClassifier

#model evaluation
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix,recall_score,precision_score,f1_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#read the dataset
data = pd.read_json('News_Category_Dataset_v2.json',lines=True) # lines for avoid the trailing error
column = data.columns
column

Index(['category', 'headline', 'authors', 'link', 'short_description', 'date'], dtype='object')

## Data Inspection


In [3]:
data.head(5)

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
#delete the dublicate values
data.duplicated().sum() # count the total duplicate samples

13

In [5]:
#there can be dublicate of author names so check for the dublicate headline and short discription
data.duplicated(subset=['headline', 'short_description']).sum()

488

In [6]:
# Drop duplicates with empty headline or empty description
data.drop_duplicates(subset=['headline', 'short_description'],inplace=True,keep='last')

In [7]:
data.describe()

C:\Users\Thor\AppData\Local\Temp/ipykernel_28576/3175101911.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe()


,category,headline,authors,link,short_description,date
count,200365,200365,200365,200365,200365,200365
unique,41,199344,27986,200341,178353,2309
top,POLITICS,Sunday Roundup,,https://www.huffingtonpost.comhttp://testkitch...,,2012-07-18 00:00:00
freq,32719,90,36514,2,19611,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2018-05-26 00:00:00


#Handling null data


In [8]:
data.isnull().sum()

category             0
headline             0
authors              0
link                 0
short_description    0
date                 0
dtype: int64

In [9]:
# there is no null value instead of null they are blank so we need to check for the blank placess and delete that
data[data['headline'] == '']

,category,headline,authors,link,short_description,date
86893,RELIGION,,"Matthew Fox, ContributorRadical theologian Mat...",https://www.huffingtonpost.com/entry/post_9671...,Let everyone not wrapped in tired and disprove...,2015-06-30
113471,QUEER VOICES,,"Gary Snyder, ContributorWriter and Media Strat...",https://www.huffingtonpost.com/entry/beverly-h...,,2014-08-28


In [10]:
# drop the blank values
headline_blank = data['headline'] == ''
data = data[~headline_blank]
print("THe length of the datset ------>",data.shape)

THe length of the datset ------> (200363, 6)


In [11]:
#drop the blank short describtion column
description_blank = data['short_description']==''
print("the lenth of the blank description samples----->",len(data[description_blank]))
data = data[~description_blank]
print("THe length of the datset ---------------------->",data.shape)

the lenth of the blank description samples-----> 19610
THe length of the datset ----------------------> (180753, 6)


## Balance datasets and process text

### Balance the Category Data


In [12]:
category = data['category'].value_counts()
list(category.index)

['POLITICS',
 'WELLNESS',
 'ENTERTAINMENT',
 'STYLE & BEAUTY',
 'TRAVEL',
 'PARENTING',
 'FOOD & DRINK',
 'QUEER VOICES',
 'HEALTHY LIVING',
 'BUSINESS',
 'COMEDY',
 'SPORTS',
 'HOME & LIVING',
 'BLACK VOICES',
 'THE WORLDPOST',
 'WEDDINGS',
 'PARENTS',
 'DIVORCE',
 'IMPACT',
 'WOMEN',
 'CRIME',
 'MEDIA',
 'WEIRD NEWS',
 'WORLD NEWS',
 'GREEN',
 'TECH',
 'TASTE',
 'RELIGION',
 'SCIENCE',
 'MONEY',
 'STYLE',
 'ARTS & CULTURE',
 'ENVIRONMENT',
 'WORLDPOST',
 'FIFTY',
 'GOOD NEWS',
 'LATINO VOICES',
 'CULTURE & ARTS',
 'COLLEGE',
 'EDUCATION',
 'ARTS']

In [13]:
cateo_keep = (data['category'] == 'POLITICS') | (data['category'] == 'WELLNESS' )| (data['category'] == 'ENTERTAINMENT') | (data['category'] == 'TRAVEL') | \
            (data['category'] == 'STYLE & BEAUTY') | (data['category'] == 'PARENTING' )| (data['category'] == 'HEALTHY LIVING') | (data['category'] == 'QUEER VOICES') | \
              (data['category'] == 'FOOD & DRINK') | (data['category'] == 'BUSINESS' )| (data['category'] == 'COMEDY') | (data['category'] == 'PARENTS') | (data['category'] == 'SPORTS') | (data['category'] == 'HOME & LIVING' )| (data['category'] == 'BLACK VOICES')
data = data[cateo_keep]



In [14]:
cateo_keep

0         False
1          True
2          True
3          True
4          True
          ...  
200848    False
200849     True
200850     True
200851     True
200852     True
Name: category, Length: 180753, dtype: bool

In [15]:
cateo_keep = (data['category'] == 'POLITICS') | (data['category'] == 'WELLNESS' )| (data['category'] == 'ENTERTAINMENT') | (data['category'] == 'TRAVEL') | \
            (data['category'] == 'STYLE & BEAUTY') | (data['category'] == 'PARENTING' )| (data['category'] == 'HEALTHY LIVING') | (data['category'] == 'QUEER VOICES') | \
              (data['category'] == 'FOOD & DRINK') | (data['category'] == 'BUSINESS' )| (data['category'] == 'COMEDY') | (data['category'] == 'PARENTS') | (data['category'] == 'SPORTS') | (data['category'] == 'HOME & LIVING' )| (data['category'] == 'BLACK VOICES')
data = data[cateo_keep]



In [16]:
# To balance the data for better prediction, sample each of the category with 3000 samples
data_1 = data[data['category'] == 'POLITICS']
data_1 = resample(data_1, replace=False, n_samples=3000, random_state=123)
data_2 = data[data['category'] == 'WELLNESS']
data_2 = resample(data_2, replace=False, n_samples=3000, random_state=123)
data_3 = data[data['category'] == 'ENTERTAINMENT']
data_3 = resample(data_3, replace=False, n_samples=3000, random_state=123)
data_4 = data[data['category'] == 'TRAVEL']
data_4 = resample(data_4, replace=False, n_samples=3000, random_state=123)
data_5 = data[data['category'] == 'STYLE & BEAUTY']
data_5 = resample(data_5, replace=False, n_samples=3000, random_state=123)
data_6 = data[data['category'] == 'PARENTING']
data_6 = resample(data_6, replace=False, n_samples=3000, random_state=123)
data_7 = data[data['category'] == 'HEALTHY LIVING']
data_7 = resample(data_7, replace=False, n_samples=3000, random_state=123)
data_8 = data[data['category'] == 'QUEER VOICES']
data_8 = resample(data_8, replace=False, n_samples=3000, random_state=123)
data_9 = data[data['category'] == 'FOOD & DRINK']
data_9 = resample(data_9, replace=False, n_samples=3000, random_state=123)
data_10 = data[data['category'] == 'BUSINESS']
data_10 = resample(data_10, replace=False, n_samples=3000, random_state=123)
data_11 = data[data['category'] == 'COMEDY']
data_11 = resample(data_11, replace=False, n_samples=3000, random_state=123)
data_12= data[data['category'] == 'PARENTS']
data_12 = resample(data_12, replace=False, n_samples=3000, random_state=123)
data_13= data[data['category'] == 'SPORTS']
data_13 = resample(data_13, replace=False, n_samples=3000, random_state=123)
data_14 = data[data['category'] == 'HOME & LIVING']
data_14 = resample(data_14, replace=False, n_samples=3000, random_state=123)
data_15 = data[data['category'] == 'BLACK VOICES']
data_15 = resample(data_15, replace=False, n_samples=3000, random_state=123)

cato_list = [data_1 , data_2 , data_3 , data_4 ,data_5 , data_6 , data_7, data_8 , data_9 , data_10, data_11 , data_12 , data_13, data_14 ,data_15]

data = pd.concat(cato_list)
data['category'].value_counts()

POLITICS          3000
WELLNESS          3000
ENTERTAINMENT     3000
TRAVEL            3000
STYLE & BEAUTY    3000
PARENTING         3000
HEALTHY LIVING    3000
QUEER VOICES      3000
FOOD & DRINK      3000
BUSINESS          3000
COMEDY            3000
PARENTS           3000
SPORTS            3000
HOME & LIVING     3000
BLACK VOICES      3000
Name: category, dtype: int64

In [17]:
data.head(5)

,category,headline,authors,link,short_description,date
77976,POLITICS,Republican Activists Increasingly Favor Outsid...,Mark Blumenthal and Ariel Edwards-Levy,https://www.huffingtonpost.com/entry/republica...,"Is this election an anomaly, or will the party...",2015-10-09
46953,POLITICS,A Crooked Wall Street Billionaire Is Crafting ...,Zach Carter,https://www.huffingtonpost.com/entry/a-crooked...,Hoo boy.,2016-09-26
356,POLITICS,GOP Candidate Set Up Trust That Pays His Kids ...,Mary Papenfuss,https://www.huffingtonpost.com/entry/vickers-c...,"Even his own mom calls him a bigot, but former...",2018-05-19
106292,POLITICS,"The U.S. Isn't in the Business of Bigotry, Are...","Terry O'Neill, ContributorPresident, National ...",https://www.huffingtonpost.com/entry/the-us-do...,"Birth control is life-saving, life-affirming h...",2014-11-19
67266,POLITICS,Bernie Sanders On 'SNL' Is Everything We Hoped...,Paige Lavender,https://www.huffingtonpost.com/entry/bernie-sa...,Amazing.,2016-02-07


### Combining the column


In [18]:
data['text'] = data['headline']+'-'+data['short_description']

In [19]:
data.head(5)

,category,headline,authors,link,short_description,date,text
77976,POLITICS,Republican Activists Increasingly Favor Outsid...,Mark Blumenthal and Ariel Edwards-Levy,https://www.huffingtonpost.com/entry/republica...,"Is this election an anomaly, or will the party...",2015-10-09,Republican Activists Increasingly Favor Outsid...
46953,POLITICS,A Crooked Wall Street Billionaire Is Crafting ...,Zach Carter,https://www.huffingtonpost.com/entry/a-crooked...,Hoo boy.,2016-09-26,A Crooked Wall Street Billionaire Is Crafting ...
356,POLITICS,GOP Candidate Set Up Trust That Pays His Kids ...,Mary Papenfuss,https://www.huffingtonpost.com/entry/vickers-c...,"Even his own mom calls him a bigot, but former...",2018-05-19,GOP Candidate Set Up Trust That Pays His Kids ...
106292,POLITICS,"The U.S. Isn't in the Business of Bigotry, Are...","Terry O'Neill, ContributorPresident, National ...",https://www.huffingtonpost.com/entry/the-us-do...,"Birth control is life-saving, life-affirming h...",2014-11-19,"The U.S. Isn't in the Business of Bigotry, Are..."
67266,POLITICS,Bernie Sanders On 'SNL' Is Everything We Hoped...,Paige Lavender,https://www.huffingtonpost.com/entry/bernie-sa...,Amazing.,2016-02-07,Bernie Sanders On 'SNL' Is Everything We Hoped...


### Cleaning Text by remove stop words, all lower case, remove url, keep only characters 

In [20]:
#drop the other columns
data.drop(['authors','headline','short_description'],axis=1,inplace=True)

In [21]:

def clean_text(text):
    text = text.lower()                                  # lower-case all characters
    text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
    text =  re.sub(r'http\S+', '',text)                  # remove urls
    text =  re.sub(r'pic.\S+', '',text) 
    text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')   # remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i)>2])
    text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    return text

In [22]:

# Cleaning the text
data['text'] = data['text'].apply(clean_text)

In [23]:

# shuffle the dataset 
from sklearn.utils import shuffle
data = shuffle(data)
data.reset_index(inplace=True, drop=True) 
data.head(4)
print(len(data),type(data))

45000 <class 'pandas.core.frame.DataFrame'>


# Adding weather category to dataset

In [24]:
import os
# Adding the weather related data to the initial dataframe
for filename in os.listdir("weather"):
    with open(os.path.join("weather", filename), 'r',encoding='utf-8') as f:
        text = f.read()
        entry = {'text':text,'category':'weather'}
        data = data.append(entry, ignore_index = True)

In [25]:
len(data)

45050

# Splitting the data and creating dense vector data


In [26]:
# X: Defines the descriptions 
X = data['text']
label = LabelEncoder()
# y: has the labels of the corresponding category
y = label.fit_transform(data['category'])

#split the train and test dataset
corpus_train,corpus_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state=2)

In [27]:
#Creating a Tfidf vectorizer object

vecto =  TfidfVectorizer(stop_words='english',max_df = 0.99,min_df=10,
                                   ngram_range=(1, 2),lowercase=True, max_features=5000)
vecto = vecto.fit(corpus_train)

# Converting sparse data to dense data
X_train = vecto.transform(corpus_train).toarray()
X_test = vecto.transform(corpus_test).toarray()


# Chi square test

In [28]:
X_names = vecto.get_feature_names() # Names of the word in the document
y_names = label.inverse_transform(y_train) # All the category names in the train
p_value_limit = 0.85 # P value for chi square test


dtf_features = pd.DataFrame()
for cat in np.unique(y_names):
    chi2_val , p = sklearn.feature_selection.chi2(X_train, y_names==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()

d:\python\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [39]:
for cat in np.unique(y_names):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(dtf_features[dtf_features["y"]==cat]))
    print("  . top features:", ",".join(
dtf_features[dtf_features["y"]==cat]["feature"].values[:10]))
    print(" ")

# BLACK VOICES:
  . selected features: 757
  . top features: african,african american,african americans,atlanta,black,black history,black lives,black man,black men,black panther
 
# BUSINESS:
  . selected features: 813
  . top features: bank,banks,billion,business,businesses,ceo,ceos,companies,company,consumers
 
# COMEDY:
  . selected features: 560
  . top features: bee,cat,cats,colbert,comedian,comedians,comedy,conan,corden,donald
 
# ENTERTAINMENT:
  . selected features: 763
  . top features: actor,actress,album,amy,amy schumer,bachelor,box office,film,game thrones,gomez
 
# FOOD & DRINK:
  . selected features: 810
  . top features: bacon,baked,baking,barbecue,bbq,beef,beer,best,bread,breakfast
 
# HEALTHY LIVING:
  . selected features: 765
  . top features: anxiety,cancer,depression,diet,disease,gps,gps guide,health,illness,mental
 
# HOME & LIVING:
  . selected features: 830
  . top features: apartment,appraises,architects,bathroom,bedroom,clean,cleaning,color,country living,craft

# Fitting the vectorizer with new vocabulary

In [29]:
# New model after with feature test 
vecto_2 = TfidfVectorizer(vocabulary=X_names) 
vecto_2.fit(corpus_train)
X_train_2 = vecto_2.transform(corpus_train)
dic_vocabulary = vecto.vocabulary_

# Ensemble Learning

In [30]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [31]:
# Creating the supervised model objects and MultiLayerPerceptron model object
svm = SVC(kernel='poly',degree=2)
mlp = MLPClassifier(max_iter=1000,activation='relu',hidden_layer_sizes = (100,),early_stopping=True)
mnb = MultinomialNB()

In [35]:
# Creating a ensemble learner classifier object

voting = VotingClassifier(estimators= [('svm',svm),('mnb',mnb),('mlp',mlp)],voting='hard')
voting.fit(X_train_2[:4000],y_train[:4000]) # Usually this training takes atleast 6 horurs so reduce the training data if needed
# voting.score(X_test,y_test)

VotingClassifier(estimators=[('svm', SVC(degree=2, kernel='poly')),
                             ('mnb', MultinomialNB()),
                             ('mlp',
                              MLPClassifier(early_stopping=True,
                                            max_iter=1000))])

# Event prediction for new document

In [36]:
new_doc = """
"Be confident in your heritage. Be confident in your blackness," President Barack Obama told graduates and their families at Howard University's 2016 Commencement Ceremony. It was one of many moments in a speech that honored the achievements of black folks — many Howard alumni — and called on graduates to get and stay politically active. His speech was met with laughter, generous applause, and largely positive reviews. Paul Holston, editor-in-chief of Howard's student newspaper The Hilltop, wrote that Obama's address was "strong, eloquent, and inspirational," and would "go down as one of the most significant moments in Howard University's history."

Where Does The 'Pull Up Your Pants' School Of Black Politics Come From?
CODE SWITCH
Where Does The 'Pull Up Your Pants' School Of Black Politics Come From?
Howard students weren't the only ones cheering over the speech. Janell Ross at The Washington Post lauded Obama's call for "empathy and [an] expanded moral imagination" as one of the few surprising and thought-provoking messages that graduates will receive this season. On Twitter, Slate writer Jamelle Bouie called the speech "a great mediation on democracy AND a celebration of black life." Mathew Rodriguez at Mic described Obama's speech as "one of the best and blackest he's given."

Melissa Harris-Perry, editor-at-large of Elle, wrote that Obama's speech was remarkable in its treatment of gender as well as race, and proved "that he is our most black, feminist president to date" by highlighting the genius of black women like Lorraine Hansberry, Harriet Tubman, Fannie Lou Hamer and Zora Neale Hurston:

"Once again, [Obama] put black women at the very center of the stories he told and the lessons he imparted. As he warmed up, he jokingly referred to 'Shonda Rhimes owning Thursday night' and 'Beyonce running the world.' They were casual references, not central themes of his talk, but even here he deployed two boss black women as representatives of black excellence and achievement."

The tone surprised some African-Americans who had been critical of what they see as the president's habit of talking down to primarily black crowds. The last time Obama spoke at an HBCU's commencement was at Morehouse College in 2013, where he was criticized for promoting a finger-wagging brand of respectability politics with remarks like these:

"Just as Morehouse has taught you to expect more of yourselves, inspire those who look up to you to expect more of themselves. We know that too many young men in our community continue to make bad choices. And I have to say, growing up, I made quite a few myself. Sometimes I wrote off my own failings as just another example of the world trying to keep a black man down. I had a tendency sometimes to make excuses for me not doing the right thing. But one of the things that all of you have learned over the last four years is there's no longer any room for excuses."

Ta-Nehisi Coates responded to that speech by calling out what he saw as the double standard Obama used in addressing African-Americans. In a piece called "How the Obama Administration Talks to Black America," Coates wrote that the president acts like someone "who sees holding African Americans to a standard of individual responsibility as part of his job. This is not a role Barack Obama undertakes with other communities."

Some observers who were worried that the president might affect the same scolding posture at Howard were pleased, if not entirely won over. Michael P. Jeffries, over at The Boston Globe, said that the Howard speech was more earnest in its depiction of structural inequality:

"...noting that a black woman is only paid 66 cents for every $1 earned by an equally qualified white man, and that mass incarceration has exploded since [Obama's] college days. In one striking passage, he reminded the audience: 'We have cousins and uncles and brothers and sisters who we remember were just as smart and just as talented as we were, but somehow got ground down by structures that are unfair and unjust.' Rather than individual failings, Obama shifted responsibility away from black families and toward the institutions that produce black suffering."

Still, Jeffries observes "how much further [Obama] has to go" to fairly depict race in America. In his speech, Obama praises Black Lives Matter activist Brittany Packnett as someone who broke with the orthodoxy of her movement to enact change. But, according to Jeffries:

"What Obama left out is that Packnett is not an anomaly among Black Lives Matter leadership. Protesters have interrupted campaign events for Hillary Clinton, Bernie Sanders, and Donald Trump, but activist DeRay McKesson certainly believes in voting: He ran for mayor of Baltimore. The Chicago-based Black Youth Project has protested mayor Rahm Emanuel and held rallies in the name of Rekia Boyd and other victims of police violence. The organization has also published research reports, and its directors have worked with several well established and likeminded groups, including the NAACP.

So, in many respects, Black Lives Matter is already living out the charge put forth by the president."

Clarence B. Jones, at HuffPost Black Voices, wrote that "the content of what President Obama said, and the way in which he spoke it were engaging; at times, powerfully moving." But he agreed with Jeffries's sentiment that the president undersold the accomplishments of the BLM movement when he chastised young people for not turning out to vote in midterm elections:

"It is not enough to patronizingly lecture that 'the perfect' 'should not be the enemy of the good or the better.' He should have not just singled out Brittany Packnett, a leader in the Black Lives Matter Movement, for praise in meeting with him and other establishment political leaders. He should have said, flat out, like 'Straight Outta Compton,' that leaders of the Movement, like Dr. King earlier, had forced America's conscience to confront the reality of successive police shootings of black men, in several circumstances where the use of non-lethal force appeared to be an available option to effect an arrest.

In effect, President Obama should have acknowledged that he AND ALL America owe a debt of gratitude to the courage and leadership of the Black Lives Matter movement in highlighting the apparent systemic racism in our criminal justice system when applied to African-Americans in several or our communities, nationwide."

BLM activists themselves had some things to say about Obama's speech. DeRay McKesson tweeted his approval of the president's remarks, but also warned about oversimplifying the message:

"Obama's commencement speech at HowardU today was important, as we both reflected on the past in blackness and our future. Obama's speech was complicated, as he noted the role of compromise in the work of progress, while maintains a commitment to one's values. Obama also rightly noted that awareness is not the win, but is the initial work that creates space for later wins. [He] also noted the work of activists like [Brittany Packnett], noting that we will have to be intentional in how we change systems/structures. In many ways, this speech echoed themes he offered when [she and I] met with him a month ago. He is becoming more explicit re: discussing blackness."

McKesson continued:

"We protest to change the world, not to continue protesting until the end of time. Awareness must lead to work focused on concrete solutions. Obama's focus on voting was not an indictment of the movement, of protest, or of organizing. Don't reduce his speech to this stale reading."

Still, others weren't blown away by the speech. Maya Rhodan at Time magazine described Obama's Howard speech as another replica from the "mold he often leans on in remarks to black audiences." She offered up the president's time-tested speech recipe: "a nod to our nation's racial history, a pit-stop on his presidency, and a call to pay it forward."

In Commencement Speech, Obama Advises Howard University Grads On Creating Change
THE TWO-WAY
In Commencement Speech, Obama Advises Howard University Grads On Creating Change
Over at The Guardian, Steven W. Thrasher reflected that "part of Obama's genius as our first black president is that he can provoke so many responses...even in the course of a single speech." But that genius is complicated. Thrasher was inspired by Obama's address until "respectability politics started to creep in," and the president began urging the crowd to empathize with "the refugee, the immigrant, the rural poor, the transgender person and yes, the middle-aged white guy who you may think has all the advantages, but over the last several decades has seen his world upended by economic and cultural and technological change and feels powerless to stop it." Here's more from Thrasher:

"Why did the nation's first black president feel the need to equate the transgender person who can't use the bathroom in North Carolina, and the unfairly maligned immigrant with that 'middle-aged white guy'?

Who feels so threatened by the 'cultural' change of living under a black president and living under conditions a little more like those black Americans have endured for hundreds of years that he's likely voting for Donald Trump?

Who isn't losing all of his white privilege, because he still has a black president telling black grads to get in his head?
ABOUT THE DIGITAL COLLECTION
The American Slavery Documents Collection contains an assortment of legal and personal documents related to slavery in the United States. Nearly all of the documents are singular and otherwise unrelated to the other, but as a composite, the collection brings to light the details of the lives and deaths of free and enslaved African Americans during the Antebellum and early Reconstruction Eras. The type of materials include bills of sale, manumission papers, emancipation notes, bonds, auction notices and other assorted items. The documents represent nearly all of the states of the American south including: North Carolina, Virginia, Alabama, Georgia, and Mississippi, but a few documents are from northern states like New York and New Jersey.

 Some materials and descriptions may include offensive content. More info

ITEMS IN THIS DIGITAL COLLECTION (202)
Thumbnail
Bill of sale for one slave: Beth, 8 year old girl; sold by Lawrence Lancaster to Thomas Cook for 32, 10 shillings
Thumbnail
Bill of sale for an enslaved person, Leander, from N.C. Trowbridge to E.H. Simmons
Thumbnail
Bill of sale, Court of Common Plea, Court at Camden
Thumbnail
Bill of sale for one slave girl: 11 y.o., 32 pounds and 10 shillings, from Absalom Lancaster to Thomas Cook
Thumbnail
Bill of sale for one slave, from Aroh Currison to Thomas Cook, N.C.
Thumbnail
Bill of sale for three slaves: Toby, James, and Jacob, from Zachariah Flurry to James Prown, S.C.
Browse all 202 Items »

SOURCE COLLECTION
This digital collection comprises selected materials from the following archival collection at David M. Rubenstein Rare Book & Manuscript Library:

American slavery documents collection 1757-1924 and undated
Collection #RL.11093 | 2.0 Linear Feet; 2 boxes; 1 oversize folder

ABSTRACT
Collection of manuscript items relating to American slavery assembled over a number of decades by the staff of the David M. Rubenstein Rare Book and Manuscript Library at Duke University. Collection contains items documenting the sales, escapes, and emancipations of enslaved people from colonial times through the Civil War, and to a lesser extent, materials relating to slavery in the United States dating from the post-emancipation period.

Collection Guide »
Panton, Jonathan (2020) "Pro-Slavery Rhetoric in the Constitution," Res Publica -
Journal of Undergraduate Research: Vol. 25
Available at: https://digitalcommons.iwu.edu/respublica/vol25/iss1/9
This Editorial is protected by copyright and/or related rights. It has been brought to you by Digital
Commons @ IWU with permission from the rights-holder(s). You are free to use this material in any
way that is permitted by the copyright and related rights legislation that applies to your use. For
other uses you need to obtain permission from the rights-holder(s) directly, unless additional rights
are indicated by a Creative Commons license in the record and/ or on the work itself. This material
has been accepted for inclusion by editorial board of Res Publica at Illinois Wesleyan University. For
more information, please contact digitalcommons@iwu.edu.
©Copyright is owned by the author of this document.
Pro-Slavery Rhetoric in the Constitution oric in the Constitution
Abstract
Prior to the Thirteenth Amendment, the United States Constitution did not merely allow slavery: the
document protected slavery. At the time of the Declaration of Independence in 1776, the Continental
Congress vigorously debated the “peculiar institution” of slavery. The lofty ideals of the Declaration
proclaimed the “inalienable rights, including life, liberty, and the pursuit of happiness.” By the time the
Constitution was adopted in 1789, the delegates all but abandoned such lofty goals to set forth an
administrative guide for government. The Constitution was a pro-slavery document because of the ⅗
clause and it enabled slavery. 
"""

## Predict process


In [38]:
doc_test = vecto_2.transform([new_doc]).toarray()

return_pred = voting.predict(doc_test)

print(' The event category is :' + label.inverse_transform(return_pred)[0]) #### Result of the prediction

 The event category is :BLACK VOICES
